<a href="https://colab.research.google.com/github/sundarp17/sundar_info5731_fall2020/blob/master/project/train/train4_unbalanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import pandas as pd

train = pd.read_csv(r'/content/train_data4.csv')
train.head()

,Unnamed: 0,Sentence,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Others
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [78]:
train['Target'].unique()

array(['Invalid', 'Others', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [79]:
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [80]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

(3476, 6427)


In [82]:
#using smote and stochastic gradient boosting
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)
sgd = SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(X_train, Y_train)
pred_sgd = sgd.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_sgd,y_val))
print(classification_report(y_val,pred_sgd))

Accuracy 0.6758849557522124
                  precision    recall  f1-score   support

        Analysis       0.50      0.40      0.44       127
      Conclusion       0.63      0.62      0.63        42
           Facts       0.77      0.88      0.82       487
         Invalid       0.51      0.43      0.47       128
           Issue       0.50      0.57      0.53        49
Rule/Law/Holding       0.59      0.31      0.41        71

        accuracy                           0.68       904
       macro avg       0.58      0.54      0.55       904
    weighted avg       0.66      0.68      0.66       904



In [83]:
def frequency_table(data):
    frequencytable = {}
    for key in data:
        if key in frequencytable:
            frequencytable[key] += 1
        else:
            frequencytable[key] = 1
    return frequencytable

frequency_table(train['Target'])



{'Analysis': 520,
 'Conclusion': 179,
 'Facts': 1769,
 'Invalid': 550,
 'Issue': 218,
 'Rule/Law/Holding': 240}

In [84]:
test = pd.read_csv(r'/content/test_data.csv')
test.head()

,Unnamed: 0,Sentence,Target
0,2173,Darden Sr. testified â€œI saw he was staggerin...,Facts
1,466,Additional moral justification may have been d...,Analysis
2,525,"85, 22 S.W. 140, wherein the accused was actin...",Rule/Law/Holding
3,2199,Under a proper instruction from the court on m...,Facts
4,79,He is eligible for leaves to renew contacts wi...,Facts


In [85]:
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [86]:
from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
test['Sentence'] = test['Sentence'].apply(lambda x: TextBlob(x).words)
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [87]:
test['Sentence'].head()

0    darden sr testified âœi saw staggering â prett...
1    additional moral justification may due complai...
2    85 22 sw 140 wherein accused acting upon advic...
3    proper instruction court murder malice murder ...
4                   eligible leaf renew contact family
Name: Sentence, dtype: object

In [88]:
len(test['Sentence'])

521

In [89]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [90]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = sgd.predict(x_un1)
unlabel_1['Predicted']=pred_unlabel_1
unlabel_1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,Sentence,Target,Predicted
0,2173,darden sr testified âœi saw staggering â prett...,Facts,Facts
1,466,additional moral justification may due complai...,Analysis,Analysis
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding,Invalid
3,2199,proper instruction court murder malice murder ...,Facts,Rule/Law/Holding
4,79,eligible leaf renew contact family,Facts,Facts


In [16]:
test_target = test['Target']

In [ ]:
len(test_target)

521

In [91]:
del unlabel_1['Target']

In [92]:
unlabel_1 = unlabel_1.rename(columns={'Predicted':'Target'})
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
2,525,85 22 sw 140 wherein accused acting upon advic...,Invalid
3,2199,proper instruction court murder malice murder ...,Rule/Law/Holding
4,79,eligible leaf renew contact family,Facts


In [93]:
frame_1 = [train,unlabel_1]
train_1 = pd.concat(frame_1)
len(train_1)

3577

In [94]:
x_train_1 = tfidf_vect.transform(train_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train_1['Target'],test_size=0.27,random_state=42)
sgd = SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(X_train, Y_train)
pred_sgd = sgd.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_sgd,y_val))
print(classification_report(y_val,pred_sgd))

Accuracy 0.6614906832298136
                  precision    recall  f1-score   support

        Analysis       0.47      0.37      0.41       146
      Conclusion       0.66      0.61      0.64        44
           Facts       0.76      0.85      0.80       503
         Invalid       0.51      0.47      0.49       150
           Issue       0.56      0.60      0.58        48
Rule/Law/Holding       0.55      0.40      0.46        75

        accuracy                           0.66       966
       macro avg       0.58      0.55      0.56       966
    weighted avg       0.65      0.66      0.65       966



In [95]:
del unlabel_2['Target']
del unlabel_3['Target']
del unlabel_4['Target']
del unlabel_5['Target']

In [96]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = sgd.predict(x_un2)
unlabel_2['Target']=pred_unlabel_2
unlabel_2.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,Sentence,Target
101,1207,examination occurred night middle may testifie...,Facts
102,3400,undisputed testimony reflects appellant drivin...,Facts
103,2072,appellant testify offer evidence behalf,Facts
104,517,56 uniform act regulating traffic highway gist...,Facts
105,906,agreed accompany gave pawn ticket money reclai...,Facts


In [97]:
frame_2 = [train_1,unlabel_2]
train_2 = pd.concat(frame_2)
len(train_2)

3677

In [98]:
x_train_2 = tfidf_vect.transform(train_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train_2['Target'],test_size=0.27,random_state=42)
sgd = SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(X_train, Y_train)
pred_sgd = sgd.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_sgd,y_val))
print(classification_report(y_val,pred_sgd))

Accuracy 0.6817724068479355
                  precision    recall  f1-score   support

        Analysis       0.52      0.44      0.47       149
      Conclusion       0.66      0.57      0.61        47
           Facts       0.77      0.89      0.83       528
         Invalid       0.54      0.46      0.50       145
           Issue       0.60      0.45      0.52        55
Rule/Law/Holding       0.46      0.32      0.38        69

        accuracy                           0.68       993
       macro avg       0.59      0.52      0.55       993
    weighted avg       0.66      0.68      0.67       993



In [99]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
x_un3.shape
pred_unlabel_3 = sgd.predict(x_un3)
unlabel_3['Target']=pred_unlabel_3
unlabel_3.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,Sentence,Target
201,1882,find evidence amply sufficient sustain juryâs ...,Analysis
202,964,action show thatâ,Facts
203,1417,certification attending physician shown filed ...,Facts
204,2932,testified agent bland first left automobile wa...,Facts
205,1584,officer testified could smell odor alcohol app...,Facts


In [100]:
frame_3 = [train_2,unlabel_3]
train_3 = pd.concat(frame_3)
len(train_3)
x_train_3 = tfidf_vect.transform(train_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train_3['Target'],test_size=0.25,random_state=42)
sgd = SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(X_train, Y_train)
pred_sgd = sgd.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_sgd,y_val))
print(classification_report(y_val,pred_sgd))

Accuracy 0.6592592592592592
                  precision    recall  f1-score   support

        Analysis       0.46      0.43      0.44       141
      Conclusion       0.72      0.48      0.58        54
           Facts       0.77      0.86      0.81       494
         Invalid       0.47      0.45      0.46       132
           Issue       0.57      0.52      0.54        56
Rule/Law/Holding       0.46      0.34      0.39        68

        accuracy                           0.66       945
       macro avg       0.58      0.51      0.54       945
    weighted avg       0.65      0.66      0.65       945



In [101]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
x_un4.shape
pred_unlabel_4 = sgd.predict(x_un4)
unlabel_4['Target']=pred_unlabel_4
unlabel_4.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,Sentence,Target
301,266,section 6 pl,Rule/Law/Holding
302,3469,amended answer appellant alleged bond invalid ...,Invalid
303,1471,evidence sufficient support conviction appella...,Analysis
304,1990,case cited note 19,Invalid
305,2757,july 8 1960 order entered revoking said probat...,Facts


In [102]:
frame_4 = [train_3,unlabel_4]
train_4 = pd.concat(frame_4)
len(train_4)
x_train_4 = tfidf_vect.transform(train_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train_4['Target'],test_size=0.2,random_state=42)
sgd = SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(X_train, Y_train)
pred_sgd = sgd.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_sgd,y_val))
print(classification_report(y_val,pred_sgd))

Accuracy 0.6868556701030928
                  precision    recall  f1-score   support

        Analysis       0.49      0.43      0.46       112
      Conclusion       0.71      0.63      0.67        43
           Facts       0.80      0.89      0.84       409
         Invalid       0.51      0.47      0.49       116
           Issue       0.50      0.38      0.43        39
Rule/Law/Holding       0.51      0.46      0.48        57

        accuracy                           0.69       776
       macro avg       0.59      0.54      0.56       776
    weighted avg       0.67      0.69      0.68       776



In [103]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])
x_un5.shape
pred_unlabel_5 = sgd.predict(x_un5)
unlabel_5['Target']=pred_unlabel_5
unlabel_5.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,Sentence,Target
401,2979,overrule appellantâs contention court erred pe...,Conclusion
402,2135,state regarded authority case,Analysis
403,1591,testimony adduced appellant mother lillie mae ...,Facts
404,658,appellant needle mark inside left elbow,Facts
405,2562,analysis content 3 capsule chemist kenneth and...,Facts


In [108]:
frame_5 = [train_4,unlabel_5]
train_5 = pd.concat(frame_5)
len(train_5)
x_train_5 = tfidf_vect.transform(train_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train_5['Target'],test_size=0.18,random_state=2)
sgd = SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(X_train, Y_train)
pred_sgd = sgd.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_sgd,y_val))
print(classification_report(y_val,pred_sgd))

Accuracy 0.7
                  precision    recall  f1-score   support

        Analysis       0.49      0.50      0.49        98
      Conclusion       0.67      0.54      0.60        37
           Facts       0.77      0.89      0.83       378
         Invalid       0.63      0.50      0.55       105
           Issue       0.63      0.55      0.59        44
Rule/Law/Holding       0.68      0.36      0.47        58

        accuracy                           0.70       720
       macro avg       0.64      0.56      0.59       720
    weighted avg       0.69      0.70      0.69       720



In [109]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = sgd.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6084452975047985
